In [2]:
import pandas as pd

In [ ]:
# Load the dataset
df = pd.read_csv('Input_Data/Customer_Service_Training_Dataset.csv')

In [6]:
# Drop the 'tags' column
df.drop(columns=['tags'], inplace=True)

# Remove any duplicate rows to avoid training bias
df.drop_duplicates(inplace=True)

# Remove missing values
df.dropna(inplace=True)

In [9]:
pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder

In [16]:
# For finetuning with a BERT model, we need to convert the 'utterance' column to tokens and then convert 'intent' and 'category' to categorical encodings
# For now, these are tokenised to work with the "bert-base-cased", but this can easily be changed in the future if we change models.

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
df['tokenized'] = df['utterance'].apply(lambda x: tokenizer.tokenize(x))

intent_encoder = LabelEncoder()
df['intent_label'] = intent_encoder.fit_transform(df['intent'])

category_encoder = LabelEncoder()
df['category_label'] = category_encoder.fit_transform(df['category'])

In [27]:
# Now we encode the user messages
max_length = 128

def encode_text(text):
    encoded = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
    return encoded['input_ids'][0], encoded['attention_mask'][0]

df[['input_ids', 'attention_mask']] = df['utterance'].apply(lambda x: encode_text(x)).apply(pd.Series)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6539 entries, 0 to 6538
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   utterance       6539 non-null   object
 1   intent          6539 non-null   int8  
 2   category        6539 non-null   int8  
 3   tokenized       6539 non-null   object
 4   intent_label    6539 non-null   int64 
 5   category_label  6539 non-null   int64 
 6   input_ids       6539 non-null   object
 7   attention_mask  6539 non-null   object
dtypes: int64(2), int8(2), object(4)
memory usage: 319.4+ KB


In [28]:
# We can save the final DataFrame to a new CSV file, and then use it later for training
df.to_csv('Cleaned_Data/Customer_Service_Training_Dataset_Final.csv', index=False)